In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
import re
from datetime import datetime
import numpy as np
from matplotlib import rcParams
import matplotlib.pyplot as plt
rcParams['font.sans-serif'] = 'Source Han Sans K'
pd.options.display.max_rows = 20
pd.options.display.min_rows = 10

In [2]:
def get_soup(url, encoding):
    req = requests.get(url)
    req.encoding = encoding
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [3]:
def 상장공모_얻기(url):
    bs = get_soup(url, 'cp949')    
    속성 = {'bgcolor': "#FFFFFF", 'colspan': "3"}
    for bs in bs.find_all('td', attrs=속성):
        if '신주모집' in bs.text:
            상장공모 = bs.text
            break
    상장공모 = [a.strip().replace(',', '') for a in 상장공모.split('/')]        
    상장공모 = [re.sub('\d+%', '', a) for a in 상장공모]        
    상장공모 = [int(re.search('\d+', a).group(0)) for a in 상장공모]
    if len(상장공모) == 1:
        상장공모.append(0)
    return 상장공모

In [4]:
def 배정수량_얻기(url):
    bs = get_soup(url, 'cp949')    
    인수회사 = {
        'bgcolor': "#C2D2DF",
        'border': "0",
        'cellpadding': "0",
        'cellspacing': "1",
        'width': "100%",
    }
    temp = bs.find_all('table', attrs=인수회사)
    for a in temp:
        if '인수회사' in a.text:
            break
    배정수량 = [x.text.strip() for x in a.find_all('tr')]
    배정수량 = [a.split('\n') for a in 배정수량]
    배정수량 = 배정수량[1:]
    return 배정수량

In [5]:
def 공모데이터_얻기(url):
    bs = get_soup(url, 'cp949')
    기업개요 = bs.find_all('table', {'summary': '기업개요'})[0]
    기업개요 = 기업개요.find_all('td')
    result = {}
    for i, a in enumerate(기업개요):
        if i % 2 == 0:
            key = a.text.strip()
        else:
            value = a.text.strip()
            result[key] = value    
    매출액 = result['매출액'].replace('(백만원)', '').replace(',', '').replace('-', '0')
    result['매출액'] = int(매출액)
    순이익 = result['순이익'].replace('(백만원)', '').replace(',', '')
    result['순이익'] = 순이익
    자본금 = result['자본금'].replace('(백만원)', '').replace(',', '').replace('-', '0')
    result['자본금'] = int(자본금)
    시장구분 = result['시장구분']  
    공모정보 = bs.find_all('table', {'summary': '공모정보'})[0]
    공모정보 = 공모정보.find_all('td')
    for i, a in enumerate(공모정보):
        if i % 2 == 0:
            key = a.text.strip()
        else:
            value = a.text.strip()
            result[key] = value
    if len(result['주간사'].split(',')) == 1:
        key = key.replace('\xa0', '').split('/')
        result['배정주식수'] = get_int(key[0])
        result['청약한도'] = get_int(key[1])            
    희망공모가액 = result['희망공모가액'].replace('원', '').replace(',', '').replace('-', '0')
    희망공모가액 = [int(a) for a in 희망공모가액.split('~')]
    result['희망공모가액'] = 희망공모가액
    확정공모가 = result['확정공모가'].replace('원', '').replace(',', '').replace('-', '0')
    result['확정공모가'] = int(확정공모가)
    총공모주식수 = result['총공모주식수'].replace('주', '').replace(',', '').replace('-', '0')
    result['총공모주식수'] = int(총공모주식수)
    공모금액 = result['공모금액'].replace('(백만원)', '').replace(',', '').replace('-', '0')
    result['공모금액'] = int(공모금액)
    try:
        청약경쟁률 = result['청약경쟁률'].split(':')[0].replace(',', '')
        result['청약경쟁률'] = float(청약경쟁률)
    except:
        pass
    공모청약일정 = bs.find_all('table', {'summary': '공모청약일정'})[0]
    공모청약일정 = [a.text.strip() for a in 공모청약일정.find_all('td')]
    추출 = ['수요예측일', '공모청약일', '상장일', '기관경쟁률', '의무보유확약']
    for i, a in enumerate(공모청약일정):
        if a in 추출:
            result[a] = 공모청약일정[i + 1].replace('\t', '').replace('\xa0', '')
    try:
        기관경쟁률 = result['기관경쟁률'].split(':')[0].replace(',', '')
        result['기관경쟁률'] = float(기관경쟁률)                
    except:
        pass        
    result['의무보유확약'] = round(float(result['의무보유확약'].replace('%', '')) * 0.01, 4)
    if len(result['주간사'].split(',')) > 1:
        배정수량들 = 배정수량_얻기(url)
        for 배정수량 in 배정수량들:
            배정수량 = [a.replace(',', '').replace('주', '').replace(' ', '') for a in 배정수량]
            result[배정수량[0]] = {'배정주식수': 배정수량[1], '청약한도': 배정수량[2]}
    try:
        상장공모 = 상장공모_얻기(url)
        result['신주모집'] = 상장공모[0]
        result['구주매출'] = 상장공모[1]
    except:
        pass
    
    return result

In [6]:
def get_int(string):
    result = ''
    for s in string:
        if s.isdigit():
            result += s
    return result

- 종목별 공모정보 주소 얻기

In [7]:
page_url = 'http://www.38.co.kr/html/fund/index.htm?o=nw&page={}'
base_url = 'http://www.38.co.kr/html/fund/?o=v&no={}&l='
regex = re.compile("no=\d+") # <a href="./?o=v&amp;no=1578&amp;l=">
last_page = 55
result = {}
for page_num in range(1, last_page + 1):
    url = page_url.format(page_num)
    bs = get_soup(url, 'cp949')
    tds = bs.find_all('td')
    for td in tds:
        links = td.find_all('a', {'href': regex})
        if links:
            break
    for link in links:
        if not 'html' in link.attrs['href']:
            링크 = base_url.format(get_int(link.attrs['href']))
            result[link.text] = 링크

- 종목별 공모데이터 얻기

In [8]:
결과 = {}
i = 1
for key, val in result.items():
    결과[key] = 공모데이터_얻기(val)
    print(f'{i} / {len(result)}')
    i += 1

1 / 1100
2 / 1100
3 / 1100
4 / 1100
5 / 1100
6 / 1100
7 / 1100
8 / 1100
9 / 1100
10 / 1100
11 / 1100
12 / 1100
13 / 1100
14 / 1100
15 / 1100
16 / 1100
17 / 1100
18 / 1100
19 / 1100
20 / 1100
21 / 1100
22 / 1100
23 / 1100
24 / 1100
25 / 1100
26 / 1100
27 / 1100
28 / 1100
29 / 1100
30 / 1100
31 / 1100
32 / 1100
33 / 1100
34 / 1100
35 / 1100
36 / 1100
37 / 1100
38 / 1100
39 / 1100
40 / 1100
41 / 1100
42 / 1100
43 / 1100
44 / 1100
45 / 1100
46 / 1100
47 / 1100
48 / 1100
49 / 1100
50 / 1100
51 / 1100
52 / 1100
53 / 1100
54 / 1100
55 / 1100
56 / 1100
57 / 1100
58 / 1100
59 / 1100
60 / 1100
61 / 1100
62 / 1100
63 / 1100
64 / 1100
65 / 1100
66 / 1100
67 / 1100
68 / 1100
69 / 1100
70 / 1100
71 / 1100
72 / 1100
73 / 1100
74 / 1100
75 / 1100
76 / 1100
77 / 1100
78 / 1100
79 / 1100
80 / 1100
81 / 1100
82 / 1100
83 / 1100
84 / 1100
85 / 1100
86 / 1100
87 / 1100
88 / 1100
89 / 1100
90 / 1100
91 / 1100
92 / 1100
93 / 1100
94 / 1100
95 / 1100
96 / 1100
97 / 1100
98 / 1100
99 / 1100
100 / 1100
101 / 11

- 종목별 공모데이터 최종 취합

In [9]:
page_url = 'http://www.38.co.kr/html/fund/index.htm?o=nw&page={}'
last_page = 55
df = []
for page_num in range(1, last_page + 1):
    url = page_url.format(page_num)
    bs = get_soup(url, 'cp949')
    bs = bs.find_all('table', {'summary': '신규상장종목'})[0]
    if page_num == 1:
        header = []
        for a in bs.find_all('th'):
            if a.text != '':
                header.append(a.text)
        df.append(header)
    temp = []
    for tbody in bs.find_all('tbody'):
        for td in tbody.find_all('td'):
            if td.text != '':
                temp.append(td.text.replace('\xa0', ''))
    for i in range(0, len(temp), 9):
        df.append(temp[i:i+9])
df = DataFrame(df[1:], columns=df[0])        
df['신규상장일'] = df['신규상장일'].map(lambda x: datetime.strptime(x, '%Y/%m/%d'))
df.columns = [column.replace('(원)', '') for column in df.columns]
columns = ['신규상장일', '기업명', '공모가', '시초가', '첫날종가', '현재가']
df =  df[columns]
df = df.set_index('신규상장일')
df = df.replace('-', 0)
df = df.replace('예정', 0)
columns = ['공모가', '시초가', '첫날종가', '현재가']
for column in columns:
    df[column] = df[column].str.replace(',', '')
df = df.fillna(0)
df['희망공모가_하한'] = df.기업명.map(lambda x: 결과[x]['희망공모가액'][0])
df['희망공모가_상한'] = df.기업명.map(lambda x: 결과[x]['희망공모가액'][1])
df['확정공모가'] = df.기업명.map(lambda x: 결과[x]['확정공모가'])
df['기관경쟁률'] = df.기업명.map(lambda x: 결과[x]['기관경쟁률'])
df['청약경쟁률'] = df.기업명.map(lambda x: 결과[x]['청약경쟁률'])
df['의무보유확약'] = df.기업명.map(lambda x: 결과[x]['의무보유확약'])
df['주간사'] = df.기업명.map(lambda x: 결과[x]['주간사'])
df['총공모주식수'] = df.기업명.map(lambda x: 결과[x]['총공모주식수'])
df['신주모집'] = df.기업명.map(lambda x: 결과[x].get('신주모집', 0))
df['구주매출'] = df.기업명.map(lambda x: 결과[x].get('구주매출', 0))
df['공모금액(백만원)'] = df.기업명.map(lambda x: 결과[x].get('공모금액'))
df['매출액(백만원)'] = df.기업명.map(lambda x: 결과[x].get('매출액'))
df['순이익(백만원)'] = df.기업명.map(lambda x: 결과[x].get('순이익'))
df['자본금(백만원)'] = df.기업명.map(lambda x: 결과[x].get('자본금'))
df['시장구분'] = df.기업명.map(lambda x: 결과[x].get('시장구분'))

In [10]:
df

기업명    공모가    시초가   첫날종가    현재가  희망공모가_하한  희망공모가_상한  \
신규상장일                                                                       
2024-05-09          이노그리드      0      0      0  37500     29000     35000   
2024-05-07             코칩      0      0      0  35500     11000     14000   
2024-05-02            민테크      0      0      0      0      6500      8500   
2024-04-03        아이엠비디엑스      0      0      0      0      7700      9900   
2024-03-27        하나스팩32호   2000      0      0      0      2000      2000   
...                   ...    ...    ...    ...    ...       ...       ...   
2012-04-30       에스비아이모기지   7000   6300   5360      0      7700      9200   
2012-04-05  코오롱패션머티리얼(유가)  14500  20000  23000      0     12000     14500   
2012-03-21           빛샘전자   4400   8800  10100      0      4000      4400   
2012-02-23        휴비스(유가)  12200  12250  11550      0     11700     13200   
2012-02-21        사람인에이치알   5000  10000  11500      0      4000      5000   

            확정공모가    기관경쟁률    청약경쟁률  의무보유확약     주간사    총공모주식수     신주모집  \
신규상장일                                                                    
2024-05-09      0                       0.0  한국투자증권    600000   600000   
2024-05-07      0                       0.0  한국투자증권   1500000  1500000   
2024-05-02      0                       0.0    KB증권   3000000  3000000   
2024-04-03      0                       0.0  미래에셋증권   2500000  2500000   
2024-03-27   2000  1247.72   2389.8     0.0    하나증권   3000000  3000000   
...           ...      ...      ...     ...     ...       ...      ...   
2012-04-30   7000              1.81     0.0  하나대투증권   7123000  3799000   
2012-04-05  14500            691.76     0.0  우리투자증권   3000000  3000000   
2012-03-21   4400           1000.47     0.0  미래에셋증권    780000   780000   
2012-02-23  12200             25.32     0.0    대우증권  16405000  5250000   
2012-02-21   5000            1057.8     0.0    현대증권   2430070  2430070   

                구주매출  공모금액(백만원)  매출액(백만원) 순이익(백만원)  자본금(백만원) 시장구분  
신규상장일                                                              
2024-05-09         0      17400     16189     388       1963  코스닥  
2024-05-07         0      16500     38750    5702       3479  코스닥  
2024-05-02         0      19500     11915   -7056       2128  코스닥  
2024-04-03         0      19250      2625  -10436       1141  코스닥  
2024-03-27         0       6000         0       -         20  코스닥  
...              ...        ...       ...      ...       ...  ...  
2012-04-30   3324000      49861    156124   26705          0       
2012-04-05         0      43500    516245   18679      74483  거래소  
2012-03-21         0       3432     29170    4147       2110  코스닥  
2012-02-23  11155000     200141   1125337   24027     143750  거래소  
2012-02-21         0      12150     31362    2029       3184  코스닥  

[1100 rows x 20 columns]

In [11]:
df.describe()

희망공모가_하한       희망공모가_상한          확정공모가       의무보유확약        총공모주식수  \
count    1100.000000    1100.000000    1100.000000  1100.000000  1.100000e+03   
mean    13849.531818   16495.100000   15441.736364     0.089127  4.398250e+06   
std     23537.272186   28514.412933   27010.644028     0.149602  6.966977e+06   
min      1200.000000    1500.000000       0.000000     0.000000  2.500000e+05   
25%      2000.000000    2000.000000    2000.000000     0.000000  1.400000e+06   
50%      8700.000000   10000.000000   10000.000000     0.013850  2.545660e+06   
75%     16000.000000   19500.000000   18000.000000     0.113200  5.000000e+06   
max    400000.000000  498000.000000  498000.000000     0.861600  9.700000e+07   

               신주모집          구주매출     공모금액(백만원)      매출액(백만원)      자본금(백만원)  
count  1.100000e+03  1.100000e+03  1.100000e+03  1.100000e+03  1.100000e+03  
mean   4.058502e+06  3.320643e+05  7.374074e+04  1.065919e+05  1.153711e+04  
std    6.690380e+06  1.623118e+06  4.480745e+05  4.878332e+05  8.259373e+04  
min    0.000000e+00  0.000000e+00  1.000000e+03  0.000000e+00  0.000000e+00  
25%    1.213975e+06  0.000000e+00  1.000000e+04  0.000000e+00  6.797500e+02  
50%    2.340200e+06  0.000000e+00  1.805800e+04  1.899300e+04  2.815500e+03  
75%    4.500000e+06  0.000000e+00  3.705900e+04  5.947425e+04  5.400750e+03  
max    9.700000e+07  2.469816e+07  1.275000e+07  8.310181e+06  2.375501e+06

In [12]:
df.describe(include=['O'])

기업명   공모가   시초가  첫날종가   현재가 기관경쟁률 청약경쟁률     주간사 순이익(백만원)  시장구분
count    1100  1100  1100  1100  1100  1100  1100    1100     1100  1100
unique   1100   216   597   678   532   788  1050     222      776     5
top     이노그리드  2000  2000     0     0              한국투자증권       -    코스닥
freq        1   286    35    31   484   309     5     129      314   985

In [13]:
df['순이익(백만원)'] = df['순이익(백만원)'].replace('- ', np.NaN)
df['기관경쟁률'] = df['기관경쟁률'].replace('', np.NaN)
df['청약경쟁률'] = df['청약경쟁률'].replace('', np.NaN)
df['시초가'] = df['시초가'].replace(0, np.NaN)
df['첫날종가'] = df['첫날종가'].replace(0, np.NaN)

In [14]:
df.dropna(axis=0, inplace=True)
df

기업명    공모가    시초가   첫날종가    현재가  희망공모가_하한  희망공모가_상한  확정공모가  \
신규상장일                                                                          
2024-03-13     오상헬스케어  20000  33250  29350  23600     13000     15000  20000   
2024-03-07    케이엔알시스템  13500  37000  27050  25200      9000     11000  13500   
2024-02-23         코셈  16000  42000  25550  24450     12000     14000  16000   
2024-02-23       이에이트  20000  26150  22600  22600     14500     18500  20000   
2024-02-22       케이웨더   7000  19920  16600   9580      4800      5800   7000   
...               ...    ...    ...    ...    ...       ...       ...    ...   
2014-08-01      덕신하우징  13000  21650  20950      0      9600     11000  13000   
2014-07-30      창해에탄올   8300  16600  14600      0      6000      6900   8300   
2014-07-25       윈하이텍   8300  13300  15250      0      6600      7500   8300   
2014-07-24      아진엑스텍   7000   7900   7060      0      5000      6000   7000   
2014-07-22  화인베스틸(유가)   4700   4490   5160      0      4500      5100   4700   

              기관경쟁률    청약경쟁률  의무보유확약            주간사   총공모주식수     신주모집  구주매출  \
신규상장일                                                                         
2024-03-13   993.21  2126.13  0.0285         NH투자증권   990000   990000     0   
2024-03-07   873.20  2266.72  0.3186  DB금융투자,NH투자증권  2104000  2104000     0   
2024-02-23  1267.60   2518.4  0.1004           키움증권   600000   600000     0   
2024-02-23   630.64   381.16  0.0235         한화투자증권  1130000  1130000     0   
2024-02-22  1362.50  1988.83  0.0393         NH투자증권  1000000  1000000     0   
...             ...      ...     ...            ...      ...      ...   ...   
2014-08-01   650.72   899.07  0.5986           현대증권  2000000  2000000     0   
2014-07-30   578.20   675.79  0.5950         우리투자증권  1900871  1900871     0   
2014-07-25   444.10   589.58  0.5560         우리투자증권  2640000  2640000     0   
2014-07-24   478.65   876.27  0.0000         신한금융투자  1000000  1000000     0   
2014-07-22    85.45   248.12  0.0000           현대증권  6433560  6433560     0   

            공모금액(백만원)  매출액(백만원) 순이익(백만원)  자본금(백만원) 시장구분  
신규상장일                                                    
2024-03-13      19800    193893   40661       7052  코스닥  
2024-03-07      28404     51182     994       1086  코스닥  
2024-02-23       9600     12525    1716       2830  코스닥  
2024-02-23      22600       309   -8112       4787  코스닥  
2024-02-22       7000     17399     709       4969  코스닥  
...               ...       ...      ...       ...  ...  
2014-08-01      26000    104434   11718       4006  코스닥  
2014-07-30      15777     69032   14887       3087  코스닥  
2014-07-25      21912     57400    7122       3300  코스닥  
2014-07-24       7000     18913    3278       2815  코스닥  
2014-07-22      30238    224271   16948      16050  거래소  

[691 rows x 20 columns]

In [15]:
df['의무보유확약비율'] = df['의무보유확약']*100
df['구주매출비율'] = df['구주매출']/df['총공모주식수']*100
df['구주매출비율'] = df['구주매출비율'].round(2)

In [16]:
df

기업명    공모가    시초가   첫날종가    현재가  희망공모가_하한  희망공모가_상한  확정공모가  \
신규상장일                                                                          
2024-03-13     오상헬스케어  20000  33250  29350  23600     13000     15000  20000   
2024-03-07    케이엔알시스템  13500  37000  27050  25200      9000     11000  13500   
2024-02-23         코셈  16000  42000  25550  24450     12000     14000  16000   
2024-02-23       이에이트  20000  26150  22600  22600     14500     18500  20000   
2024-02-22       케이웨더   7000  19920  16600   9580      4800      5800   7000   
...               ...    ...    ...    ...    ...       ...       ...    ...   
2014-08-01      덕신하우징  13000  21650  20950      0      9600     11000  13000   
2014-07-30      창해에탄올   8300  16600  14600      0      6000      6900   8300   
2014-07-25       윈하이텍   8300  13300  15250      0      6600      7500   8300   
2014-07-24      아진엑스텍   7000   7900   7060      0      5000      6000   7000   
2014-07-22  화인베스틸(유가)   4700   4490   5160      0      4500      5100   4700   

              기관경쟁률    청약경쟁률  ...   총공모주식수     신주모집  구주매출  공모금액(백만원)  \
신규상장일                         ...                                      
2024-03-13   993.21  2126.13  ...   990000   990000     0      19800   
2024-03-07   873.20  2266.72  ...  2104000  2104000     0      28404   
2024-02-23  1267.60   2518.4  ...   600000   600000     0       9600   
2024-02-23   630.64   381.16  ...  1130000  1130000     0      22600   
2024-02-22  1362.50  1988.83  ...  1000000  1000000     0       7000   
...             ...      ...  ...      ...      ...   ...        ...   
2014-08-01   650.72   899.07  ...  2000000  2000000     0      26000   
2014-07-30   578.20   675.79  ...  1900871  1900871     0      15777   
2014-07-25   444.10   589.58  ...  2640000  2640000     0      21912   
2014-07-24   478.65   876.27  ...  1000000  1000000     0       7000   
2014-07-22    85.45   248.12  ...  6433560  6433560     0      30238   

            매출액(백만원)  순이익(백만원)  자본금(백만원) 시장구분  의무보유확약비율 구주매출비율  
신규상장일                                                           
2024-03-13    193893    40661       7052  코스닥      2.85    0.0  
2024-03-07     51182      994       1086  코스닥     31.86    0.0  
2024-02-23     12525     1716       2830  코스닥     10.04    0.0  
2024-02-23       309    -8112       4787  코스닥      2.35    0.0  
2024-02-22     17399      709       4969  코스닥      3.93    0.0  
...              ...       ...       ...  ...       ...    ...  
2014-08-01    104434    11718       4006  코스닥     59.86    0.0  
2014-07-30     69032    14887       3087  코스닥     59.50    0.0  
2014-07-25     57400     7122       3300  코스닥     55.60    0.0  
2014-07-24     18913     3278       2815  코스닥      0.00    0.0  
2014-07-22    224271    16948      16050  거래소      0.00    0.0  

[691 rows x 22 columns]

In [17]:
df['순이익(백만원)'] = df['순이익(백만원)'].astype(int)
df['공모가'] = df['공모가'].astype(int)
df['시초가'] = df['시초가'].astype(int)
df['첫날종가'] = df['첫날종가'].astype(int)
df['청약경쟁률'] = df['청약경쟁률'].astype(float)

In [18]:
df.describe()

공모가            시초가           첫날종가      희망공모가_하한  \
count     691.000000     691.000000     691.000000     691.00000   
mean    21573.241679   30159.645441   29454.283647   19348.82055   
std     30835.762278   43223.787229   41250.920335   27470.05171   
min      1300.000000    1350.000000    1150.000000    1200.00000   
25%      9000.000000   11625.000000   11605.000000    8200.00000   
50%     15000.000000   20000.000000   19500.000000   13000.00000   
75%     25000.000000   33000.000000   32500.000000   21800.00000   
max    498000.000000  597000.000000  505000.000000  400000.00000   

            희망공모가_상한          확정공모가        기관경쟁률         청약경쟁률      의무보유확약  \
count     691.000000     691.000000   691.000000    691.000000  691.000000   
mean    23187.279305   21573.241679   642.378524    820.378698    0.128149   
std     33084.387782   30835.762278   574.340864   3103.335493    0.159883   
min      1500.000000    1300.000000     3.290000      0.190000    0.000000   
25%     10000.000000    9000.000000   101.790000     65.350000    0.013250   
50%     15300.000000   15000.000000   510.360000    586.430000    0.070800   
75%     25200.000000   25000.000000  1089.200000   1087.520000    0.167150   
max    498000.000000  498000.000000  2269.680000  79976.000000    0.811500   

             총공모주식수          신주모집          구주매출     공모금액(백만원)      매출액(백만원)  \
count  6.910000e+02  6.910000e+02  6.910000e+02  6.910000e+02  6.910000e+02   
mean   3.443653e+06  2.916792e+06  4.594886e+05  1.018451e+05  1.454214e+05   
std    6.183113e+06  5.572278e+06  1.825145e+06  5.617732e+05  5.854050e+05   
min    2.500000e+05  0.000000e+00  0.000000e+00  2.475000e+03  0.000000e+00   
25%    1.102500e+06  1.000000e+06  0.000000e+00  1.500000e+04  1.392500e+04   
50%    1.784500e+06  1.590000e+06  0.000000e+00  2.600000e+04  3.567400e+04   
75%    3.116265e+06  2.638165e+06  1.316600e+05  4.499450e+04  8.352250e+04   
max    8.598444e+07  8.598444e+07  2.469816e+07  1.275000e+07  8.310181e+06   

           순이익(백만원)      자본금(백만원)    의무보유확약비율      구주매출비율  
count  6.910000e+02  6.910000e+02  691.000000  691.000000  
mean   8.938350e+03  1.455726e+04   12.814920    9.047931  
std    8.800867e+04  1.007445e+05   15.988284   28.653205  
min   -4.517710e+05  0.000000e+00    0.000000    0.000000  
25%    1.840000e+02  2.493500e+03    1.325000    0.000000  
50%    3.462000e+03  3.982000e+03    7.080000    0.000000  
75%    7.685500e+03  6.680500e+03   16.715000    8.450000  
max    1.904946e+06  2.375501e+06   81.150000  622.900000

In [19]:
df.describe(include=['O'])

기업명  현재가     주간사 시장구분
count      691  691     691  691
unique     691  407     180    3
top     오상헬스케어    0  한국투자증권  코스닥
freq         1  252      98  611

In [20]:
df.drop(['의무보유확약', '현재가'], axis=1, inplace=True)
df

기업명    공모가    시초가   첫날종가  희망공모가_하한  희망공모가_상한  확정공모가  \
신규상장일                                                                   
2024-03-13     오상헬스케어  20000  33250  29350     13000     15000  20000   
2024-03-07    케이엔알시스템  13500  37000  27050      9000     11000  13500   
2024-02-23         코셈  16000  42000  25550     12000     14000  16000   
2024-02-23       이에이트  20000  26150  22600     14500     18500  20000   
2024-02-22       케이웨더   7000  19920  16600      4800      5800   7000   
...               ...    ...    ...    ...       ...       ...    ...   
2014-08-01      덕신하우징  13000  21650  20950      9600     11000  13000   
2014-07-30      창해에탄올   8300  16600  14600      6000      6900   8300   
2014-07-25       윈하이텍   8300  13300  15250      6600      7500   8300   
2014-07-24      아진엑스텍   7000   7900   7060      5000      6000   7000   
2014-07-22  화인베스틸(유가)   4700   4490   5160      4500      5100   4700   

              기관경쟁률    청약경쟁률            주간사   총공모주식수     신주모집  구주매출  \
신규상장일                                                                 
2024-03-13   993.21  2126.13         NH투자증권   990000   990000     0   
2024-03-07   873.20  2266.72  DB금융투자,NH투자증권  2104000  2104000     0   
2024-02-23  1267.60  2518.40           키움증권   600000   600000     0   
2024-02-23   630.64   381.16         한화투자증권  1130000  1130000     0   
2024-02-22  1362.50  1988.83         NH투자증권  1000000  1000000     0   
...             ...      ...            ...      ...      ...   ...   
2014-08-01   650.72   899.07           현대증권  2000000  2000000     0   
2014-07-30   578.20   675.79         우리투자증권  1900871  1900871     0   
2014-07-25   444.10   589.58         우리투자증권  2640000  2640000     0   
2014-07-24   478.65   876.27         신한금융투자  1000000  1000000     0   
2014-07-22    85.45   248.12           현대증권  6433560  6433560     0   

            공모금액(백만원)  매출액(백만원)  순이익(백만원)  자본금(백만원) 시장구분  의무보유확약비율  구주매출비율  
신규상장일                                                                       
2024-03-13      19800    193893     40661      7052  코스닥      2.85     0.0  
2024-03-07      28404     51182       994      1086  코스닥     31.86     0.0  
2024-02-23       9600     12525      1716      2830  코스닥     10.04     0.0  
2024-02-23      22600       309     -8112      4787  코스닥      2.35     0.0  
2024-02-22       7000     17399       709      4969  코스닥      3.93     0.0  
...               ...       ...       ...       ...  ...       ...     ...  
2014-08-01      26000    104434     11718      4006  코스닥     59.86     0.0  
2014-07-30      15777     69032     14887      3087  코스닥     59.50     0.0  
2014-07-25      21912     57400      7122      3300  코스닥     55.60     0.0  
2014-07-24       7000     18913      3278      2815  코스닥      0.00     0.0  
2014-07-22      30238    224271     16948     16050  거래소      0.00     0.0  

[691 rows x 20 columns]

In [21]:
df['종속변수'] = np.where(df['시초가'] == df['공모가']*2, 1, 0)

In [22]:
df

기업명    공모가    시초가   첫날종가  희망공모가_하한  희망공모가_상한  확정공모가  \
신규상장일                                                                   
2024-03-13     오상헬스케어  20000  33250  29350     13000     15000  20000   
2024-03-07    케이엔알시스템  13500  37000  27050      9000     11000  13500   
2024-02-23         코셈  16000  42000  25550     12000     14000  16000   
2024-02-23       이에이트  20000  26150  22600     14500     18500  20000   
2024-02-22       케이웨더   7000  19920  16600      4800      5800   7000   
...               ...    ...    ...    ...       ...       ...    ...   
2014-08-01      덕신하우징  13000  21650  20950      9600     11000  13000   
2014-07-30      창해에탄올   8300  16600  14600      6000      6900   8300   
2014-07-25       윈하이텍   8300  13300  15250      6600      7500   8300   
2014-07-24      아진엑스텍   7000   7900   7060      5000      6000   7000   
2014-07-22  화인베스틸(유가)   4700   4490   5160      4500      5100   4700   

              기관경쟁률    청약경쟁률            주간사  ...     신주모집  구주매출  공모금액(백만원)  \
신규상장일                                        ...                             
2024-03-13   993.21  2126.13         NH투자증권  ...   990000     0      19800   
2024-03-07   873.20  2266.72  DB금융투자,NH투자증권  ...  2104000     0      28404   
2024-02-23  1267.60  2518.40           키움증권  ...   600000     0       9600   
2024-02-23   630.64   381.16         한화투자증권  ...  1130000     0      22600   
2024-02-22  1362.50  1988.83         NH투자증권  ...  1000000     0       7000   
...             ...      ...            ...  ...      ...   ...        ...   
2014-08-01   650.72   899.07           현대증권  ...  2000000     0      26000   
2014-07-30   578.20   675.79         우리투자증권  ...  1900871     0      15777   
2014-07-25   444.10   589.58         우리투자증권  ...  2640000     0      21912   
2014-07-24   478.65   876.27         신한금융투자  ...  1000000     0       7000   
2014-07-22    85.45   248.12           현대증권  ...  6433560     0      30238   

            매출액(백만원)  순이익(백만원)  자본금(백만원)  시장구분 의무보유확약비율  구주매출비율  종속변수  
신규상장일                                                                  
2024-03-13    193893     40661      7052   코스닥     2.85     0.0     0  
2024-03-07     51182       994      1086   코스닥    31.86     0.0     0  
2024-02-23     12525      1716      2830   코스닥    10.04     0.0     0  
2024-02-23       309     -8112      4787   코스닥     2.35     0.0     0  
2024-02-22     17399       709      4969   코스닥     3.93     0.0     0  
...              ...       ...       ...   ...      ...     ...   ...  
2014-08-01    104434     11718      4006   코스닥    59.86     0.0     0  
2014-07-30     69032     14887      3087   코스닥    59.50     0.0     1  
2014-07-25     57400      7122      3300   코스닥    55.60     0.0     0  
2014-07-24     18913      3278      2815   코스닥     0.00     0.0     0  
2014-07-22    224271     16948     16050   거래소     0.00     0.0     0  

[691 rows x 21 columns]

In [23]:
오늘 = datetime.today().strftime('%Y%m%d')
df.to_csv(f'신규청약_{오늘}.csv', encoding='cp949')